In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.samsung.com/in/mobile-accessories/' 
response = requests.get(url)
response.raise_for_status()  

soup = BeautifulSoup(response.text, 'html.parser')

products = []
for product in soup.find_all(class_=lambda x: x and 'product' in x.lower()):  # Adjust the selector to be more flexible
    title = product.find(class_=lambda x: x and ('title' in x.lower() or 'name' in x.lower()))
    price = product.find(class_=lambda x: x and 'price' in x.lower())
    description = product.find(class_=lambda x: x and 'description' in x.lower())
    
    product_data = {
        'Title': title.get_text(strip=True) if title else 'N/A',
        'Price': price.get_text(strip=True) if price else 'N/A',
        'Description': description.get_text(strip=True) if description else 'N/A'
    }
    
    products.append(product_data)

if products:
    df = pd.DataFrame(products)
    df.to_excel('products.xlsx', index=False)
    print('Data has been scraped and saved to products.xlsx')
else:
    print('No products found or unable to scrape the data')


Data has been scraped and saved to products.xlsx


In [3]:
df = pd.read_excel('products.xlsx')
df.head()

,Title,Price,Description
0,New & Featured,NaN,NaN
1,NaN,NaN,NaN
2,Galaxy S24 Ultra,NaN,NaN
3,NaN,NaN,NaN
4,Galaxy S24 Ultra,NaN,NaN


In [5]:
from flask import Flask, render_template, request
import pandas as pd

app = Flask(__name__)

# Load the scraped data from the Excel file
df = pd.read_excel('products.xlsx')

@app.route('/', methods=['GET', 'POST'])
def index():
    query = request.form.get('query')
    if query:
        # Search for products matching the query
        filtered_df = df[df['Title'].str.contains(query, case=False, na=False)]
    else:
        filtered_df = df

    # Convert the DataFrame to a list of dictionaries
    products = filtered_df.to_dict(orient='records')
    
    return render_template('index.html', products=products, query=query)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\DELL\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
